In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity  # Add this import
import json
import matplotlib.pyplot as plt
import pandas as pd
datafile = "./imdb_top_1000.csv"

In [19]:
df = pd.read_csv(datafile)

In [20]:
df.columns

Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [21]:
df_hotbit = pd.get_dummies(df[['Director', 'Genre', 'Star1', 'Star2', 'Star3', 'Star4']], prefix='Category')


In [22]:
df_hotbit

,Category_Aamir Khan,Category_Aaron Sorkin,Category_Abdellatif Kechiche,Category_Abhishek Chaubey,Category_Abhishek Kapoor,Category_Adam Elliot,Category_Adam McKay,Category_Aditya Chopra,Category_Aditya Dhar,Category_Akira Kurosawa,...,Category_Yûnosuke Itô,Category_Zac Mattoon O'Brien,Category_Zach Grenier,Category_Zarah Jane McKenzie,Category_Zeppo Marx,Category_Ziyi Zhang,Category_Zoe Saldana,Category_Zoë Kravitz,Category_Álvaro Guerrero,Category_Émile Vallée
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
similarity_matrix = cosine_similarity(df_hotbit)

In [24]:
similarity_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.        , 0.16666667,
        1.        ]])

In [25]:
threshold = 0.3
links = []

for i in range(len(similarity_matrix)):
    for j in range(i + 1, len(similarity_matrix)):
        if similarity_matrix[i][j] >= threshold:
            links.append({"source": i, "target": j, "weight": round(similarity_matrix[i][j],3)})

nodes_set = set()  # To store unique node IDs
for link in links:
    nodes_set.add(link["source"])
    nodes_set.add(link["target"])

nodes = []
for idx in nodes_set:
    nodes.append({"id": idx, 
                  "title": df['Series_Title'].iloc[idx],
                  "Director": df['Series_Title'].iloc[idx],
                  "Star1": df['Star1'].iloc[idx],
                  "Star2": df['Star2'].iloc[idx],
                  "Star3": df['Star3'].iloc[idx],
                  "Genre": df['Genre'].iloc[idx] ,
                  "poster":df['Poster_Link'].iloc[idx]
                 })


In [26]:
output = {
    "links": links,
    "nodes": nodes
}
with open('output.json', 'w') as f:
    json.dump(output, f, indent=4)